In [1]:
import yfinance as yf
import pandas as pd
import datetime
import utils

In [2]:
df = pd.read_csv('../datasets/stocks.csv')
stock_names = df['ticker']
df.head()

,company,stock,link,ticker
0,Tesla Inc,Tesla,https://www.investing.com/equities/tesla-motors,TSLA
1,Palantir Technologies Inc,Palantir,https://www.investing.com/equities/palantir-te...,PLTR
2,NVIDIA Corporation,NVIDIA,https://www.investing.com/equities/nvidia-corp,NVDA
3,Apple Inc,Apple,https://www.investing.com/equities/apple-compu...,AAPL
4,Meta Platforms Inc,Meta Platforms,https://www.investing.com/equities/facebook-inc,META


In [45]:
def get_stock_info(ticker, start_date, end_date):
    # Fetch stock data
    stock_data = yf.download(ticker, start=start_date, end=end_date, progress=False)
    # Extract high, low, close, and other information
    high = stock_data['High']
    low = stock_data['Low']
    close = stock_data['Close']
    volume = stock_data['Volume']
    open = stock_data['Open']    
    return high, low, close, volume, open

In [46]:
start_date = '2024-03-19'
end_date = datetime.datetime.now().strftime('%Y-%m-%d')
stock_price_dict = {}
for stock_name in stock_names:
    try:
        if stock_name == 'TMTG':
            stock_name_alias = 'TMTG-USD'
            high, low, close, volume, open = get_stock_info(stock_name_alias, start_date, end_date)
        else:
            high, low, close, volume, open = get_stock_info(stock_name, start_date, end_date)
        df = pd.DataFrame({'high': high, 'low': low, 'close': close, 'volume': volume, 'open': open})
        utils.create_path(f'../datasets/stock_data/')
        df.to_csv(f'../datasets/stock_data/{stock_name}.csv')
        stock_price_dict[stock_name] = df['close']
    except Exception as e:
        print(f"Error: {e}")
        continue


1 Failed download:
['TMTG-USD']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2024-03-19 -> 2024-03-30)')


In [49]:
column_list = list(df.index.strftime('%Y-%m-%d'))
rows_index_list = list(stock_price_dict.keys())
rows_index_list.remove('TMTG')
new_df = pd.DataFrame(index=rows_index_list, columns=column_list)

In [51]:
for stock_name in stock_price_dict.keys():
    if stock_name == 'TMTG':
        continue
    price_list = list(stock_price_dict[stock_name])
    try:
        new_df.loc[stock_name] = price_list
    except Exception as e:
        print(e)
new_df.to_csv('../datasets/stock_price.csv')

In [52]:
new_df.head()

,2024-03-19,2024-03-20,2024-03-21,2024-03-22,2024-03-25,2024-03-26,2024-03-27,2024-03-28
TSLA,171.320007,175.660004,172.820007,170.830002,172.630005,177.669998,179.830002,175.789993
PLTR,23.799999,24.57,24.49,24.18,24.51,24.889999,24.51,23.01
NVDA,893.97998,903.719971,914.349976,942.890015,950.02002,925.609985,902.5,903.559998
AAPL,176.080002,178.669998,171.369995,172.279999,170.850006,169.710007,173.309998,171.479996
META,496.23999,505.519989,507.76001,509.579987,503.019989,495.890015,493.859985,485.579987
